<a href="https://colab.research.google.com/github/martinbrough/colab-test/blob/master/twitterarchive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Twitter News Summarizer

This notebook gets tweets from a list, finds the most popular external links and then summarises them


In [0]:
# install any libraries not included with Colab
!pip install newspaper3k

In [0]:
# imports
import tweepy
from tweepy import OAuthHandler
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import urllib.request
import requests
import re
import newspaper
from newspaper import Article
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Log into twitter and get a dataframe with tweets

In [0]:
# Twitter Credentials
twitter_key = 'mzKL70g7HoAmJ84PWxdBcd0qV'
twitter_secret_key = 'QZRXdlFKEuCRo5MwyS1OmdlZJjTNXTJLEFHrUD1T6PCzarUUou'
access_token = '355045555-BW5a4OVBx488dWchB3GSE4e2BfzzA5myvMY7jHI9'
access_token_secret = 'CCpKwnNrxpEjUtRnfbCQCi6nISpgBU9aXsy0K4oC9GyEK'



In [0]:
# Authenticate
auth = tweepy.OAuthHandler(twitter_key, twitter_secret_key)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)


In [0]:
# Get tweets
tweets = tweepy.Cursor(api.list_timeline, owner_screen_name="@martinbrough", slug="ESG", since_id = 1247884835441405957, include_entities=True).items(100)

In [0]:
#Set up the dataframe with the tweets and add a blank column ready for the expended external links
tweet_columns = ['AuthorName', 'AuthorScreenName', 'TweetDate', 'TweetId', 'TweetText', 'RetweetStatus','RetweetCount','InReplyTo', 'Urls']
tweet_list = [[tweet.author.name, tweet.author.screen_name, tweet.created_at, tweet.id, tweet.text, tweet.retweeted,tweet.retweet_count, tweet.in_reply_to_screen_name, tweet.entities['urls']] for tweet in tweets]
tweet_df = pd.DataFrame(data=tweet_list, columns=tweet_columns)
tweet_df['Link'] = ''


In [0]:
#Get the urls and populate the 'Link' column with any expanded external links
for i in range(0,len(tweet_df.index)):
# for i in range(324,380):
  links = tweet_df['Urls'][i]
  actual_link = ''
  if len(links)>0:
    link = links[0]
    expanded_url = link['expanded_url']
    if str(expanded_url).find('https://twitter.com/') == -1:
      try:
        r= requests.head(expanded_url, timeout=10)
        if r.status_code in range (200,300):
          actual_link = r.url
        elif r.status_code in range (300,400):
          actual_link = r.headers['location']
        else:
          actual_link = expanded_url
      except requests.exceptions.Timeout as e:
        actual_link = expanded_url
        print(e)
      print('Tweet ' + str(i) + ' url ' + str(expanded_url) + ' link is ' + str(actual_link))
      tweet_df.loc[i, 'Link'] = actual_link
    else:
      print('Tweet ' + str(i) + ' url ' + str(expanded_url) + ' is a twitter link.')


Tweet 1 url https://twitter.com/i/web/status/1247885358194294786 is a twiiter link.
Tweet 2 url https://twitter.com/i/web/status/1247885319707353088 is a twiiter link.
Tweet 3 url https://twitter.com/i/web/status/1247885169261858816 is a twiiter link.
Tweet 4 url https://reut.rs/2UTfcmg link is https://www.reuters.com/article/us-tiffany-m-a-lvmh-idUSKBN21Q1RL?taid=5e8dd72f9a7fcd0001c4c4d0&utm_campaign=trueAnthem:+Trending+Content&utm_medium=trueAnthem&utm_source=twitter
Tweet 5 url https://twitter.com/i/web/status/1247885047899668483 is a twiiter link.


In [0]:
tweet_df

,AuthorName,AuthorScreenName,TweetDate,TweetId,TweetText,RetweetStatus,RetweetCount,InReplyTo,Urls,Link
0,Michael S. Taylor,mtaylor_nz,2020-04-08 13:54:07,1247885436116058113,"RT @wallaceme: Amid all the worrying news, thi...",False,18204,None,[],
1,Green Party Ireland,greenparty_ie,2020-04-08 13:53:48,1247885358194294786,"On Intl #TravellerAndRomaDay2020, we're callin...",False,1,None,"[{'url': 'https://t.co/EdHfe96LI3', 'expanded_...",
2,Eurelectric,eurelectric,2020-04-08 13:53:39,1247885319707353088,With the public consultation for the Alternati...,False,0,None,"[{'url': 'https://t.co/b4XIhaxod3', 'expanded_...",
3,ACEA,ACEA_eu,2020-04-08 13:53:03,1247885169261858816,#RoadSafety fact: #speeding is the main cause ...,False,0,None,"[{'url': 'https://t.co/wGcv8m5Pbq', 'expanded_...",
4,Reuters Business,ReutersBiz,2020-04-08 13:52:49,1247885109174255616,Tiffany says Australian regulators seek more t...,False,1,None,"[{'url': 'https://t.co/J4tm8hqKFK', 'expanded_...",https://www.reuters.com/article/us-tiffany-m-a...
5,Oxford Martin School,oxmartinschool,2020-04-08 13:52:34,1247885047899668483,"In Feb, @WHO said that the #coronavirus #pande...",False,0,None,"[{'url': 'https://t.co/M16sx4yvTr', 'expanded_...",
6,Robert Jenrick,RobertJenrick,2020-04-08 13:52:31,1247885033576099841,RT @RobertJenrick: To the Jewish community thr...,False,51,None,[],


Consolidate the dataframe to find the most shared external links, then get the news headlines and summaries



In [0]:
news_df = tweet_df.groupby('Link').agg({'AuthorName': 'count', 'RetweetCount':sum})
news_df.sort_values(by=['AuthorName'], ascending=False, inplace=True)
top_news_df = news_df.reset_index()[1:11]
top_news_df['Headline']=''
top_news_df['Date']=''
top_news_df['Summary']=''
top_news_df['Text']=''

In [0]:
for i in range(1,11):
  story_link = top_news_df['Link'][i]
  article = Article(story_link)
  article.download()
  article.parse()
  article.nlp()
  top_news_df.loc[i, 'Headline'] = article.title
  top_news_df.loc[i, 'Date'] = article.publish_date
  top_news_df.loc[i, 'Summary'] = article.summary
  top_news_df.loc[i, 'Text'] = article.text
  print('')
  print(str(i) + ': ' + article.title)  
  print(story_link) 
  print(article.summary)


1: Les conséquences, par secteur économique, du confinement des Français
https://www.lemonde.fr/politique/article/2020/04/07/les-consequences-par-secteur-economique-du-confinement-des-francais_6035832_823448.html
PHILIPPE LOPEZ / AFPLe gouvernement est encore en train de prendre la mesure de la crise économique qui accompagne et suivra la crise sanitaire du Covid-19.
« C’est dire l’ampleur du choc économique auquel nous sommes confrontés », a insisté Bruno Le Maire, alors que de nombreux secteurs sont à l’arrêt du fait du confinement.
En se basant sur des données statistiques du recensement de 2016 et sur une étude européenne de 2019 sur le télétravail, les économistes tentent d’estimer les effets du confinement sur l’emploi et la valeur ajoutée par secteur.
La France mieux équipéeEn termes de valeur ajoutée, « parmi les secteurs les plus affectés, on retrouve ceux directement affectés par les mesures de distanciation sociale, comme la culture et les loisirs (−7,7 %) et l’hôtellerie-r

In [0]:
top_news_df

,Link,AuthorName,RetweetCount,Headline,Date,Summary,Text
1,https://www.lemonde.fr/politique/article/2020/...,1,3,,,,
2,https://www.weforum.org/agenda/2020/04/outbrea...,1,19,,,,
3,https://www.spglobal.com/marketintelligence/en...,1,3,,,,
4,https://www.reuters.com/article/us-usa-stocks/...,1,6,,,,
5,https://www.reuters.com/article/us-equinor-win...,1,1,,,,
6,https://www.reuters.com/article/uk-health-coro...,1,0,,,,
7,https://www.reuters.com/article/uk-health-coro...,1,2,,,,
8,https://www.responsible-investor.com/articles/...,1,2,,,,
9,https://www.nytimes.com/2020/04/07/business/en...,1,0,,,,
10,https://www.legifrance.gouv.fr/affichTexte.do?...,1,2,,,,


Other useful stuff 


You can get the list of atributes here:
https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

this is the regular expression for getting URLS from text:
re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet.text)
